# Notebook Modules

In [5]:
%load_ext literary.module

The literary.module extension is already loaded. To reload it, use:
  %reload_ext literary.module


In [6]:
import sys
import warnings
from pathlib import Path
from ..transpile.patch import patch

In [7]:
def determine_package_name(path: Path) -> str:
    """Determine the corresponding importable name for a package directory given by
    a particular file path. Return `None` if path is not contained within `sys.path`.

    :param path: path to package
    :return:
    """
    for p in sys.path:
        # If the cwd is only on `sys.path` exactly, then it can't
        # be a submodule. However, let's first look for parent paths
        if str(path) == p:
            continue

        # Resolve path relative to `sys.path`
        try:
            relative_path = path.relative_to(p)
        except ValueError:
            continue

        return ".".join(relative_path.parts)

    # Nothing was found on `sys.path`, so we abort
    return None

In [8]:
def load_ipython_extension(ipython):
    """Load the import hook and setup the global state for the Literary extension.
    When IPython invokes this function, the determined package root path will be
    added to `sys.path`.

    :param ipython: IPython shell instance
    """    
    cwd = Path.cwd()
    
    # Ensure CWD is not on sys.path
    sys.path = [p for p in sys.path if Path(p).resolve() != cwd]

    # Identify which package this belongs to
    package = determine_package_name(cwd)    
    if package is None:
        warnings.warn(f"Couldn't determine the package name for the current working directory {cwd}. "
                      f"This might be because the current project has not been installed in editable mode.")
        
        
    # Set `__package__` for consumer notebook
    ipython.user_ns.update(
        {
            "__package__": determine_package_name(cwd),
            "patch": patch,
        }
    )